# Full 3D Model 

In [12]:
import sympy as sp
import numpy as np
from sympy import Matrix, symbols, sin, cos, pi, simplify, pprint, init_printing

# init_printing(use_unicode=True)

### All symbols used / symbolic variables

In [3]:
# Define symbolic variables
t = sp.Symbol('t', real=True)  # time

# Chassis state variables
x, y, z = sp.symbols('x y z', real=True, cls=sp.Function)
roll, pitch, yaw = sp.symbols('roll pitch yaw', real=True, cls=sp.Function)

# Define functions of time for chassis position and orientation
x = x(t)
y = y(t)
z = z(t)
roll = roll(t)
pitch = pitch(t)
yaw = yaw(t)

# Define derivatives
x_dot = sp.diff(x, t)
y_dot = sp.diff(y, t)
z_dot = sp.diff(z, t)
roll_dot = sp.diff(roll, t)
pitch_dot = sp.diff(pitch, t)
yaw_dot = sp.diff(yaw, t)

# Second derivatives
x_ddot = sp.diff(x_dot, t)
y_ddot = sp.diff(y_dot, t)
z_ddot = sp.diff(z_dot, t)
roll_ddot = sp.diff(roll_dot, t)
pitch_ddot = sp.diff(pitch_dot, t)
yaw_ddot = sp.diff(yaw_dot, t)

# Joint angles for left leg
theta_l1, theta_l2 = sp.symbols('theta_l1 theta_l2', real=True, cls=sp.Function)
theta_l1 = theta_l1(t)
theta_l2 = theta_l2(t)
theta_l1_dot = sp.diff(theta_l1, t)
theta_l2_dot = sp.diff(theta_l2, t)
theta_l1_ddot = sp.diff(theta_l1_dot, t)
theta_l2_ddot = sp.diff(theta_l2_dot, t)

# Joint angles for right leg
theta_r1, theta_r2 = sp.symbols('theta_r1 theta_r2', real=True, cls=sp.Function)
theta_r1 = theta_r1(t)
theta_r2 = theta_r2(t)
theta_r1_dot = sp.diff(theta_r1, t)
theta_r2_dot = sp.diff(theta_r2, t)
theta_r1_ddot = sp.diff(theta_r1_dot, t)
theta_r2_ddot = sp.diff(theta_r2_dot, t)

# Wheel angles
phi_l, phi_r = sp.symbols('phi_l phi_r', real=True, cls=sp.Function)
phi_l = phi_l(t)
phi_r = phi_r(t)
phi_l_dot = sp.diff(phi_l, t)
phi_r_dot = sp.diff(phi_r, t)
phi_l_ddot = sp.diff(phi_l_dot, t)
phi_r_ddot = sp.diff(phi_r_dot, t)

# Control inputs (torques)
tau_l1, tau_l2, tau_lw = sp.symbols('tau_l1 tau_l2 tau_lw', real=True)  # Left leg and wheel torques
tau_r1, tau_r2, tau_rw = sp.symbols('tau_r1 tau_r2 tau_rw', real=True)  # Right leg and wheel torques

# System parameters (constants)
m_c = sp.Symbol('m_c', positive=True)  # Chassis mass
m_l1, m_l2 = sp.symbols('m_l1 m_l2', positive=True)  # Left leg link masses
m_r1, m_r2 = sp.symbols('m_r1 m_r2', positive=True)  # Right leg link masses
m_lw, m_rw = sp.symbols('m_lw m_rw', positive=True)  # Wheel masses

# Inertia tensors (simplified as scalar moments of inertia for now)
I_c = sp.Symbol('I_c', positive=True)  # Chassis inertia
I_l1, I_l2 = sp.symbols('I_l1 I_l2', positive=True)  # Left leg link inertias
I_r1, I_r2 = sp.symbols('I_r1 I_r2', positive=True)  # Right leg link inertias
I_lw, I_rw = sp.symbols('I_lw I_rw', positive=True)  # Wheel inertias

# Link lengths
L_1, L_2 = sp.symbols('L_1 L_2', positive=True)  # Link lengths
r_w = sp.Symbol('r_w', positive=True)  # Wheel radius

# Gravity
g = sp.Symbol('g', positive=True)

# Define the positions of joint attachments on the chassis (in chassis frame)
lx, ly, lz = sp.symbols('lx ly lz', real=True)  # Left joint position
rx, ry, rz = sp.symbols('rx ry rz', real=True)  # Right joint position

# Define rotation matrices for chassis orientation
def Rot_x(angle):
    return Matrix([
        [1, 0, 0],
        [0, cos(angle), -sin(angle)],
        [0, sin(angle), cos(angle)]
    ])

def Rot_y(angle):
    return Matrix([
        [cos(angle), 0, sin(angle)],
        [0, 1, 0],
        [-sin(angle), 0, cos(angle)]
    ])

def Rot_z(angle):
    return Matrix([
        [cos(angle), -sin(angle), 0],
        [sin(angle), cos(angle), 0],
        [0, 0, 1]
    ])


### Calculate Positions

In [4]:

# Full rotation matrix (ZYX convention)
R_chassis = Rot_z(yaw) * Rot_y(pitch) * Rot_x(roll)

# Chassis position in world frame
p_chassis = Matrix([x, y, z])

# Calculate positions of leg joints in world frame
p_left_joint = p_chassis + R_chassis * Matrix([lx, ly, lz])
p_right_joint = p_chassis + R_chassis * Matrix([rx, ry, rz])

# Calculate positions along the left leg
p_left_link1_end = p_left_joint + Matrix([
    L_1 * sin(theta_l1) * cos(yaw),
    L_1 * sin(theta_l1) * sin(yaw),
    -L_1 * cos(theta_l1)
])

p_left_link2_end = p_left_link1_end + Matrix([
    L_2 * sin(theta_l2) * cos(yaw),
    L_2 * sin(theta_l2) * sin(yaw),
    -L_2 * cos(theta_l2)
])

p_left_wheel = p_left_link2_end + Matrix([0, 0, -r_w])

# Calculate positions along the right leg
p_right_link1_end = p_right_joint + Matrix([
    L_1 * sin(theta_r1) * cos(yaw),
    L_1 * sin(theta_r1) * sin(yaw),
    -L_1 * cos(theta_r1)
])

p_right_link2_end = p_right_link1_end + Matrix([
    L_2 * sin(theta_r2) * cos(yaw),
    L_2 * sin(theta_r2) * sin(yaw),
    -L_2 * cos(theta_r2)
])

p_right_wheel = p_right_link2_end + Matrix([0, 0, -r_w])

# Calculate centers of mass (COM) for links (assuming COM at middle of links)
p_left_link1_com = p_left_joint + 0.5 * (p_left_link1_end - p_left_joint)
p_left_link2_com = p_left_link1_end + 0.5 * (p_left_link2_end - p_left_link1_end)
p_right_link1_com = p_right_joint + 0.5 * (p_right_link1_end - p_right_joint)
p_right_link2_com = p_right_link1_end + 0.5 * (p_right_link2_end - p_right_link1_end)


### Velocity and Kinetic Energy Calculations

In [5]:

# Calculate velocities by taking time derivatives
v_chassis = Matrix([x_dot, y_dot, z_dot])
omega_chassis = Matrix([roll_dot, pitch_dot, yaw_dot])

# For each body, calculate the velocity of its COM
v_left_link1_com = sp.diff(p_left_link1_com, t)
v_left_link2_com = sp.diff(p_left_link2_com, t)
v_right_link1_com = sp.diff(p_right_link1_com, t)
v_right_link2_com = sp.diff(p_right_link2_com, t)
v_left_wheel = sp.diff(p_left_wheel, t)
v_right_wheel = sp.diff(p_right_wheel, t)

# Calculate kinetic energy terms
# Translational kinetic energy
T_chassis_trans = 0.5 * m_c * (v_chassis.dot(v_chassis))
T_left_link1_trans = 0.5 * m_l1 * (v_left_link1_com.dot(v_left_link1_com))
T_left_link2_trans = 0.5 * m_l2 * (v_left_link2_com.dot(v_left_link2_com))
T_right_link1_trans = 0.5 * m_r1 * (v_right_link1_com.dot(v_right_link1_com))
T_right_link2_trans = 0.5 * m_r2 * (v_right_link2_com.dot(v_right_link2_com))
T_left_wheel_trans = 0.5 * m_lw * (v_left_wheel.dot(v_left_wheel))
T_right_wheel_trans = 0.5 * m_rw * (v_right_wheel.dot(v_right_wheel))

# Rotational kinetic energy (simplified for now)
T_chassis_rot = 0.5 * I_c * (omega_chassis.dot(omega_chassis))
T_left_link1_rot = 0.5 * I_l1 * (theta_l1_dot**2)
T_left_link2_rot = 0.5 * I_l2 * (theta_l2_dot**2)
T_right_link1_rot = 0.5 * I_r1 * (theta_r1_dot**2)
T_right_link2_rot = 0.5 * I_r2 * (theta_r2_dot**2)
T_left_wheel_rot = 0.5 * I_lw * (phi_l_dot**2)
T_right_wheel_rot = 0.5 * I_rw * (phi_r_dot**2)

# Total kinetic energy
T = (T_chassis_trans + T_chassis_rot + 
     T_left_link1_trans + T_left_link1_rot + 
     T_left_link2_trans + T_left_link2_rot + 
     T_right_link1_trans + T_right_link1_rot + 
     T_right_link2_trans + T_right_link2_rot + 
     T_left_wheel_trans + T_left_wheel_rot + 
     T_right_wheel_trans + T_right_wheel_rot)


### Potential Energy Calculations

In [6]:

# Potential energy
V_chassis = m_c * g * z
V_left_link1 = m_l1 * g * p_left_link1_com[2]
V_left_link2 = m_l2 * g * p_left_link2_com[2]
V_right_link1 = m_r1 * g * p_right_link1_com[2]
V_right_link2 = m_r2 * g * p_right_link2_com[2]
V_left_wheel = m_lw * g * p_left_wheel[2]
V_right_wheel = m_rw * g * p_right_wheel[2]

# Total potential energy
V = V_chassis + V_left_link1 + V_left_link2 + V_right_link1 + V_right_link2 + V_left_wheel + V_right_wheel


### Non-Holonomic Constraints

In [7]:
# 1. Wheel Contact Velocities (World Frame)

# Left Wheel Contact Velocity
v_left_contact = v_left_wheel  # Velocity of the wheel center

# Right Wheel Contact Velocity
v_right_contact = v_right_wheel # Velocity of the wheel center


# 2. Define Wheel Directions (Body Frame)

#  Wheel direction vectors (in the chassis frame) -> point forward along the x-axis of the chassis

u_left_wheel = R_chassis * Matrix([1, 0, 0])   # Left wheel direction
u_right_wheel = R_chassis * Matrix([1, 0, 0])  # Right wheel direction
u_perp = R_chassis * Matrix([0,1,0]) #Perpendicular to wheel

# 3. Constraint Equations

#   a. No Sideways Slipping (dot product with perpendicular direction = 0)
f1 = v_left_contact.dot(u_perp)   # Left wheel, no lateral slip
f2 = v_right_contact.dot(u_perp)  # Right wheel, no lateral slip

#   b. Rolling Constraint (dot product with wheel direction = r * phi_dot)
f3 = v_left_contact.dot(u_left_wheel) - r_w * phi_l_dot  # Left wheel rolling
f4 = v_right_contact.dot(u_right_wheel) - r_w * phi_r_dot # Right wheel rolling

# 4. Lagrange Multipliers
lambda_1, lambda_2, lambda_3, lambda_4 = symbols('lambda_1 lambda_2 lambda_3 lambda_4')


### Lagrangian

In [8]:

# Lagrangian
L = T - V

# Define generalized coordinates and their derivatives
q = Matrix([x, y, z, roll, pitch, yaw, theta_l1, theta_l2, theta_r1, theta_r2, phi_l, phi_r])
q_dot = Matrix([x_dot, y_dot, z_dot, roll_dot, pitch_dot, yaw_dot, 
               theta_l1_dot, theta_l2_dot, theta_r1_dot, theta_r2_dot, phi_l_dot, phi_r_dot])
q_ddot = Matrix([x_ddot, y_ddot, z_ddot, roll_ddot, pitch_ddot, yaw_ddot,
                theta_l1_ddot, theta_l2_ddot, theta_r1_ddot, theta_r2_ddot, phi_l_ddot, phi_r_ddot])

# Define generalized forces (corresponding to generalized coordinates)
Q = Matrix([0, 0, 0, 0, 0, 0, tau_l1, tau_l2, tau_r1, tau_r2, tau_lw, tau_rw])

#Constraint and Lagrange Multiplier Lists
constraints = [f1, f2, f3, f4]
lambdas = [lambda_1, lambda_2, lambda_3, lambda_4]


### Equations of Motion using Euler-Lagrange equations

In [9]:

# Euler-Lagrange equations:
# d/dt(∂L/∂q̇) - ∂L/∂q = Q
# -> equations of motion in the form: M(q)q̈ + C(q,q̇)q̇ + G(q) = B·τ
# wheel no slip is non holonomic constraint

def derive_eom(L, q, q_dot, Q):
    n = len(q)
    eom = []
    
    for i in range(n):
        # Calculate ∂L/∂q̇ᵢ
        dL_dqdot = sp.diff(L, q_dot[i])
        
        # Calculate d/dt(∂L/∂q̇ᵢ)
        ddL_dtdqdot = sp.diff(dL_dqdot, t)
        
        # Calculate ∂L/∂qᵢ
        dL_dq = sp.diff(L, q[i])
        
        # Formulate equation of motion: d/dt(∂L/∂q̇ᵢ) - ∂L/∂qᵢ = Qᵢ
        eom_i = ddL_dtdqdot - dL_dq - Q[i]
        eom.append(eom_i)
    
    return Matrix(eom)


def derive_eom_with_constraints(L, q, q_dot, Q, constraints, lambdas):
    n = len(q)
    m = len(constraints)
    eom = []

    for i in range(n):
        # d/dt(dL/dq_dot)
        term1 = sp.diff(sp.diff(L, q_dot[i]), t)
        # dL/dq
        term2 = sp.diff(L, q[i])

        # Lagrange multiplier term
        lagrange_term = 0
        for j in range(m):
            lagrange_term += lambdas[j] * sp.diff(constraints[j], q[i])

        # Euler-Lagrange equation
        eq = term1 - term2 - Q[i] - lagrange_term
        eom.append(eq)

    # Add the constraint equations
    for constr in constraints:
        eom.append(constr)

    return Matrix(eom)



### Initial Constants and State to Linearize about

In [10]:
# Initial joint angles
theta_l1_0 = -pi/4
theta_l2_0 = 5 * pi/4
theta_r1_0 = -pi/4
theta_r2_0 = 5 * pi/4

# Initial Chassis Height
z0 = r_w + np.abs(L_1 * sin(theta_l1_0)) + np.abs(L_2 * sin(theta_l2_0))

eq_state = {
    x: 0, y: 0, z: z0,
    roll: 0, pitch: 0, yaw: 0,
    theta_l1: theta_l1_0, theta_l2: theta_l2_0,
    theta_r1: theta_r1_0, theta_r2: theta_r2_0,
    phi_l: 0, phi_r: 0,
    x_dot: 0, y_dot: 0, z_dot: 0,
    roll_dot: 0, pitch_dot: 0, yaw_dot: 0,
    theta_l1_dot: 0, theta_l2_dot: 0,
    theta_r1_dot: 0, theta_r2_dot: 0,
    phi_l_dot: 0, phi_r_dot: 0
}

zero_control_state = {
    tau_l1: 0, tau_l2: 0, tau_lw: 0,
    tau_r1: 0, tau_r2: 0, tau_rw: 0
}

# The z0, theta_l1_0, etc. would be determined based on the desired robot posture

# Define state vector for state-space representation
x_state = Matrix([
    x, y, z, roll, pitch, yaw,
    theta_l1, theta_l2, theta_r1, theta_r2, phi_l, phi_r,
    x_dot, y_dot, z_dot, roll_dot, pitch_dot, yaw_dot,
    theta_l1_dot, theta_l2_dot, theta_r1_dot, theta_r2_dot, phi_l_dot, phi_r_dot
])

# Input vector
u = Matrix([tau_l1, tau_l2, tau_lw, tau_r1, tau_r2, tau_rw])


### Equations of Motion

In [ ]:
# eoms = derive_eom(L, q, q_dot, Q)
eoms = derive_eom_with_constraints(L, q, q_dot, Q, constraints, lambdas)
for eq in eoms:
    pprint(eq)
    print()

# Write the equations to a latex file
with open('eoms.tex', 'w') as f:
    f.write(sp.latex(eoms))

### Linearization and Calculation of A and B Matrices

In [ ]:
def compute_eom_residuals(state_and_controls, L, q, q_dot, Q, constraints, lambdas, param_values):
    """
    Computes the residuals of the equations of motion (including constraints).
    This function is designed for use with numerical solvers.

    Args:
        state_and_controls: A NumPy array containing the state variables (q, q_dot),
                             control inputs (u), and Lagrange multipliers (lambdas).
        L: The Lagrangian.
        q: List of generalized coordinates.
        q_dot: List of generalized velocities.
        Q: List of generalized forces.
        constraints: List of constraint equations.
        lambdas: List of Lagrange multipliers.
        param_values: Dictionary mapping symbolic parameters to numerical values.

    Returns:
        A NumPy array of residuals (values of the left-hand side of the EOMs
        and constraint equations).
    """

    n = len(q)
    m = len(constraints)
    k = len(Q) - n   # Number of "control inputs"
    # Split state_and_controls
    current_q = state_and_controls[:n]
    current_q_dot = state_and_controls[n:2*n]
    current_controls =  state_and_controls[2*n : 2*n + k]
    current_lambdas = state_and_controls[2*n + k:]

    # Create substitution dictionaries
    q_subs = dict(zip(q, current_q))
    q_dot_subs = dict(zip(q_dot, current_q_dot))
    #Need to remake Q
    Q_temp = [0, 0, 0, 0, 0, 0, current_controls[0], current_controls[1], current_controls[2], current_controls[3], current_controls[4], current_controls[5]]

    Q_subs = dict(zip(Q, Q_temp))
    lambda_subs = dict(zip(lambdas, current_lambdas))

    subs_dict = {**q_subs, **q_dot_subs, **Q_subs, **lambda_subs, **param_values}

    # Evaluate equations of motion
    eom_values = [eom.subs(subs_dict) for eom in eoms]

    # Convert to NumPy array and flatten
    return np.array(eom_values, dtype=np.float64).flatten()

def compute_jacobian_numerically(func, x, *args):
    """
    Computes the Jacobian of a function numerically using central finite differences.

    Args:
        func: The function to compute the Jacobian of.
        x: The point at which to evaluate the Jacobian (NumPy array).
        *args: Additional arguments to pass to `func`.

    Returns:
        The numerically computed Jacobian matrix (NumPy array).
    """
    epsilon = 1e-8  # Step size for finite differences
    n = len(x)
    m = len(func(x, *args))  # Get number of outputs from func
    jacobian = np.zeros((m, n))

    for i in range(n):
        x_plus = x.copy()
        x_minus = x.copy()
        x_plus[i] += epsilon
        x_minus[i] -= epsilon
        f_plus = func(x_plus, *args)
        f_minus = func(x_minus, *args)
        jacobian[:, i] = (f_plus - f_minus) / (2 * epsilon)

    return jacobian
#Define function to combine states and controls for the residual
def combine_state_controls(q_vals, q_dot_vals, control_vals, lambda_vals):
    return np.concatenate([q_vals, q_dot_vals, control_vals, lambda_vals])

def split_state_controls(combined, n, k):
    q_vals = combined[:n]
    q_dot_vals = combined[n:2*n]
    control_vals = combined[2*n:2*n + k]
    lambda_vals = combined[2*n + k:]
    return q_vals, q_dot_vals, control_vals, lambda_vals
def compute_AB_matrices(L, q, q_dot, Q, constraints, lambdas, u, eq_state, param_values):
  """Computes the A and B matrices numerically."""
  n = len(q)       # number of generalized coordinates
  k = len(u)  # number of control inputs
  m = len(constraints) #number of constraints

  # Combine equilibrium state and zero controls
  eq_q = [eq_state[qi] for qi in q]   #numerical values
  eq_q_dot = [eq_state[qi] for qi in q_dot]
  eq_controls = [0] * k  # Zero controls at equilibrium
  # Initial guess for Lagrange multipliers 
  eq_lambdas = [0.0] * m

  #Initial state
  x0 = combine_state_controls(eq_q, eq_q_dot, eq_controls, eq_lambdas)

  # Find initial lambda values
  from scipy.optimize import fsolve
  # Define a function for fsolve
  def equations(initial_conditions):
      # Split initial conditions into q_ddot and lambda values
      initial_q_ddot, initial_lambdas = initial_conditions[:n], initial_conditions[n:]
      # Combine all values for subs: q, q_dot, q_ddot, lambdas, and control inputs
      combined_values = {**eq_state, **param_values}
      for i, q_dd in enumerate(q_ddot):
        combined_values[q_dd] = initial_q_ddot[i]
      for i, lm in enumerate(lambdas):
        combined_values[lm] = initial_lambdas[i]
      #also need to add zero controls
      for i, u_term in enumerate(u):
        combined_values[u_term] = 0

      # Evaluate eoms with substitutions
      evaluated_eoms = [eom.subs(combined_values) for eom in eoms]
    
      return evaluated_eoms

  # Call fsolve to find initial_q_ddot and initial_lambdas
  initial_conditions_guess = [0.0] * (n + m)  # Initial guess for q_ddot and lambdas
  solved_initial_conditions, info, ier, mesg = fsolve(equations, initial_conditions_guess, full_output=True)
  if ier != 1:
    print("Warning: fsolve did not converge:", mesg)
  eq_lambdas = solved_initial_conditions[n:] #update lambdas


  #Recalculate x0 with solved lambdas
  x0 = combine_state_controls(eq_q, eq_q_dot, eq_controls, n           )

  #Compute the A and B matrices
  A_numerical = compute_jacobian_numerically(compute_eom_residuals, x0, L, q, q_dot, Q, constraints, lambdas, param_values)
  B_numerical = A_numerical[:, 2*n:2*n + k] #B is part of the big jacobian
  A_numerical = A_numerical[:, :2*n]       #A is the rest of the big jacobian

  return A_numerical, B_numerical

# Example usage (assuming you have 'eoms', 'q', 'q_dot', 'Q', 'constraints', 'lambdas', 'u', 'eq_state', and 'param_values' defined)
# Create a dictionary for your parameter values
param_values = {
  m_c: 1.0, m_l1: 0.1, m_l2: 0.1, m_r1: 0.1, m_r2: 0.1, m_lw: 0.05, m_rw: 0.05,
  I_c: 0.1, I_l1: 0.01, I_l2: 0.01, I_r1: 0.01, I_r2: 0.01, I_lw: 0.0025, I_rw: 0.0025,
  L_1: 0.2, L_2: 0.2, r_w: 0.05, g: 9.81, lx: 0.1, ly: 0.1, lz: -0.1, rx: 0.1, ry: -0.1, rz: -0.1
}

A_num, B_num = compute_AB_matrices(L, q, q_dot, Q, constraints, lambdas, u, eq_state, param_values)
print("Numerical A Matrix:\n", A_num)
print("\nNumerical B Matrix:\n", B_num)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
def compute_A_matrix(q, q_dot, q_ddot, eoms, eq_state):
    n = len(q)
    A = sp.zeros(2*n, 2*n)
    
    # Solve for q̈ terms (rearrange eoms to get q̈ isolated)
    solved_q_ddot = sp.solve(eoms, q_ddot)
    
    # Upper-right block: derivative of q̇ wrt q̇ (identity matrix)
    for i in range(n):
        A[i, n+i] = 1
    
    # Lower blocks: derivatives of q̈ expressions
    for i in range(n):
        # Derivative with respect to q
        for j in range(n):
            A[n+i, j] = sp.diff(solved_q_ddot[q_ddot[i]], q[j]).subs(eq_state).subs(zero_control_state)
        
        # Derivative with respect to q̇
        for j in range(n):
            A[n+i, n+j] = sp.diff(solved_q_ddot[q_ddot[i]], q_dot[j]).subs(eq_state).subs(zero_control_state)
    
    return A

def compute_B_matrix(q, q_dot, q_ddot, u, eoms, eq_state):
    n = len(q)
    m = len(u)
    B = sp.zeros(2*n, m)
    
    # Solve for q̈ terms (rearrange eoms to get q̈ isolated)
    solved_q_ddot = sp.solve(eoms, q_ddot)
    
    # Lower blocks: derivatives of q̈ expressions
    for i in range(n):
        # Derivative with respect to u
        for j in range(m):
            B[n+i, j] = sp.diff(solved_q_ddot[q_ddot[i]], u[j]).subs(eq_state)
    
    return B


A_eq = compute_A_matrix(q, q_dot, q_ddot, eoms, eq_state)
B_eq = compute_B_matrix(q, q_dot, q_ddot, u, eoms, eq_state)

pprint(A_eq)
pprint(B_eq)